In [1]:
import numpy as np
import pandas as pd
import matplotlib 
from matplotlib import pyplot as plt
import string
import math

# Imports
import pandas as pd

import spacy
from spacy.tokenizer import Tokenizer
from spacy.lang.es import Spanish

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity

import pickle

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import SnowballStemmer

import PyPDF2

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jdani\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jdani\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
file = open("prueba_es.txt", "r", encoding='utf8')
lines = file.readlines()

datos = {'Documentos':lines}
df = pd.DataFrame(data = datos)
df.head(10)

,Documentos
0,BANCO DE ESPAÑA Eurosistema 23.03.2021 Nota in...


In [3]:
df['Documentos'][0]

'BANCO DE ESPAÑA Eurosistema 23.03.2021 Nota informativa Proyecciones macroeconómicas de la economía española (2021-2023) Esta nota describe los rasgos principales de las proyecciones macroeconómicas de la economía española para el período 2021-2023, que el Banco de España publica hoy en su página web (disponibles en este enlace). En relación a las proyecciones publicadas el pasado diciembre, las actuales incorporan la nueva información conocida desde entonces, lo que incluye, entre otros elementos, las estimaciones preliminares de la Contabilidad Nacional Trimestral para el cuarto trimestre de 2020 y los cambios observados en los supuestos técnicos que subyacen a la construcción de los escenarios. Las perspectivas económicas siguen sujetas a un elevado grado de incertidumbre, vinculada tanto, en el corto plazo, a la evolución de la pandemia y la velocidad del proceso de inmunización de la población como, a medio plazo, a las consecuencias económicas de la crisis. A su vez, estas últim

In [4]:
# ### Primero tokenizamos los 4 primeros documentos
# def tokenize_words(text):
#     sin_puntuacion = text.translate(str.maketrans('', '', string.punctuation))
#     tokenized_text=word_tokenize(sin_puntuacion)
#     print(tokenized_text)
#     print(type(tokenized_text))
    
#     return tokenized_text

# # df_tokenized = df.copy()
# df['Documentos'] = df['Documentos'].apply(lambda x: tokenize_words(x))
    


In [5]:
# stopword_es = nltk.corpus.stopwords.words('spanish')
# #from nltk and spacy library
# total_spanish_stopwords = list(stopwords.words("spanish")) + stopword_es

# def remove_stopwords(text):
#     text = [word for word in text if word not in stopword_es]
#     return text

# df['Sin_stopwords'] = df['Documentos'].apply(lambda x: remove_stopwords(x))

In [6]:
# df['Sin_stopwords'][0]

In [7]:
# def spanish_stemmer(x):
#     spanish_stemmer_ = SnowballStemmer(language = 'spanish')
#     for i in range(len(x)):
# #         print(i)
#         for l in range(len(x[i])):
# #             print(l)
#             x[i][l] = spanish_stemmer_.stem(x[i][l])
# #             print(x[i])
#     return x
    
# df['words_stemmed'] = spanish_stemmer(df['Sin_stopwords'])

In [8]:
# df['words_stemmed'][0]

## Empezamos a hacer el resumen

Trying with the following guide: [Click on here](https://towardsdatascience.com/text-summarization-using-tf-idf-e64a0644ace3)

In [9]:
df['Documentos'][0]

'BANCO DE ESPAÑA Eurosistema 23.03.2021 Nota informativa Proyecciones macroeconómicas de la economía española (2021-2023) Esta nota describe los rasgos principales de las proyecciones macroeconómicas de la economía española para el período 2021-2023, que el Banco de España publica hoy en su página web (disponibles en este enlace). En relación a las proyecciones publicadas el pasado diciembre, las actuales incorporan la nueva información conocida desde entonces, lo que incluye, entre otros elementos, las estimaciones preliminares de la Contabilidad Nacional Trimestral para el cuarto trimestre de 2020 y los cambios observados en los supuestos técnicos que subyacen a la construcción de los escenarios. Las perspectivas económicas siguen sujetas a un elevado grado de incertidumbre, vinculada tanto, en el corto plazo, a la evolución de la pandemia y la velocidad del proceso de inmunización de la población como, a medio plazo, a las consecuencias económicas de la crisis. A su vez, estas últim

In [10]:
sentences = sent_tokenize(df['Documentos'][0]) # NLTK function
total_documents = len(sentences)

stopword_es = nltk.corpus.stopwords.words('spanish')
#from nltk and spacy library
total_spanish_stopwords = list(stopwords.words("spanish")) + stopword_es

In [11]:
# sentences

In [12]:
def _create_frequency_matrix(sentences):
    frequency_matrix = {}
    
    stopword_es = nltk.corpus.stopwords.words('spanish')
    #from nltk and spacy library
    total_spanish_stopwords = list(stopwords.words("spanish")) + stopword_es
    stopWords = total_spanish_stopwords
    ps = SnowballStemmer(language = 'spanish')

    for sent in sentences:
        freq_table = {}
        words = word_tokenize(sent)
        for word in words:
            word = word.lower()
            word = ps.stem(word)
            if word in stopWords:
                continue

            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

        frequency_matrix[sent[:15]] = freq_table

    return frequency_matrix

def _create_tf_matrix(freq_matrix):
    tf_matrix = {}

    for sent, f_table in freq_matrix.items():
        tf_table = {}

        count_words_in_sentence = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / count_words_in_sentence

        tf_matrix[sent] = tf_table

    return tf_matrix

def _create_documents_per_words(freq_matrix):
    word_per_doc_table = {}

    for sent, f_table in freq_matrix.items():
        for word, count in f_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1

    return word_per_doc_table


def _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents):
    idf_matrix = {}

    for sent, f_table in freq_matrix.items():
        idf_table = {}

        for word in f_table.keys():
            idf_table[word] = math.log10(total_documents / float(count_doc_per_words[word]))

        idf_matrix[sent] = idf_table

    return idf_matrix


def _create_tf_idf_matrix(tf_matrix, idf_matrix):
    tf_idf_matrix = {}

    for (sent1, f_table1), (sent2, f_table2) in zip(tf_matrix.items(), idf_matrix.items()):

        tf_idf_table = {}

        for (word1, value1), (word2, value2) in zip(f_table1.items(),
                                                    f_table2.items()):  # here, keys are the same in both the table
            tf_idf_table[word1] = float(value1 * value2)

        tf_idf_matrix[sent1] = tf_idf_table

    return tf_idf_matrix

def _score_sentences(tf_idf_matrix) -> dict:
    """
    score a sentence by its word's TF
    Basic algorithm: adding the TF frequency of every non-stop word in a sentence divided by total no of words in a sentence.
    :rtype: dict
    """

    sentenceValue = {}

    for sent, f_table in tf_idf_matrix.items():
        total_score_per_sentence = 0

        count_words_in_sentence = len(f_table)
        for word, score in f_table.items():
            total_score_per_sentence += score

        sentenceValue[sent] = total_score_per_sentence / count_words_in_sentence

    return sentenceValue


def _find_average_score(sentenceValue) -> int:
    """
    Find the average score from the sentence value dictionary
    :rtype: int
    """
    sumValues = 0
    for entry in sentenceValue:
        sumValues += sentenceValue[entry]

    # Average value of a sentence from original summary_text
    average = (sumValues / len(sentenceValue))

    return average


def _generate_summary(sentences, sentenceValue, threshold):
    sentence_count = 0
    summary = ''

    for sentence in sentences:
        if sentence[:15] in sentenceValue and sentenceValue[sentence[:15]] >= (threshold):
            summary += " " + sentence
            sentence_count += 1

    return summary

In [13]:
freq_matrix= _create_frequency_matrix(sentences)

tf_matrix = _create_tf_matrix(freq_matrix)

count_doc_per_words = _create_documents_per_words(freq_matrix)

idf_matrix = _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents)

tf_idf_matrix = _create_tf_idf_matrix(tf_matrix, idf_matrix)

sentence_scores = _score_sentences(tf_idf_matrix)

threshold = _find_average_score(sentence_scores)

In [14]:
threshold

0.05373514537982196

In [15]:
summary = _generate_summary(sentences, sentence_scores, 1* threshold)
print(summary)

 BANCO DE ESPAÑA Eurosistema 23.03.2021 Nota informativa Proyecciones macroeconómicas de la economía española (2021-2023) Esta nota describe los rasgos principales de las proyecciones macroeconómicas de la economía española para el período 2021-2023, que el Banco de España publica hoy en su página web (disponibles en este enlace). A su vez, estas últimas dependerán de dos aspectos. En primer lugar, de las secuelas que la pandemia pueda dejar sobre el tejido productivo y laboral, en términos de destrucción de empresas y desempleo de larga duración. El escenario central contempla la posibilidad de que en los próximos meses surjan nuevos brotes de la enfermedad, cuya contención requeriría aplicar medidas de contención similares a las vigentes recientemente. Se proyecta, además, que la progresiva administración de las vacunas permita una retirada gradual de estas medidas, hasta su desaparición prácticamente plena a finales de 2021. El repunte reciente del IAPC tiene en gran medida una natu

In [16]:
len(summary)

1403

In [17]:
len(df['Documentos'][0])

6199

In [18]:
df.columns

Index(['Documentos'], dtype='object')

In [19]:
file_2 = open("boe.txt", "r", encoding='utf8')
lines_2 = file_2.readlines()


df = pd.DataFrame(data=[lines,lines_2],columns=['Documento'])
df['Documento'][1]

'BOE\\nBOLETÃ\x8dN OFICIAL DEL ESTADO\\nNÃºm. 62\\nViernes 12 de marzo de 2010\\nSec. I. PÃ¡g. 24831\\nI. DISPOSICIONES GENERALES\\nMINISTERIO DE EDUCACIÃ“N\\n4132\\nReal Decreto 132/2010, de 12 de febrero, por el que se establecen los requisitos\\nmÃ\xadnimos de los centros que impartan las enseÃ±anzas del segundo ciclo de la\\neducaciÃ³n infantil, la educaciÃ³n primaria y la educaciÃ³n secundaria.\\nLa Ley OrgÃ¡nica 8/1985, de 3 de julio, reguladora del Derecho a la EducaciÃ³n,\\nestablece, en su artÃ\xadculo 14, que todos los centros docentes, independientemente de su\\ntitularidad, deberÃ¡n reunir unos requisitos mÃ\xadnimos referidos a titulaciÃ³n acadÃ©mica del\\nprofesorado, relaciÃ³n numÃ©rica alumno-profesor, instalaciones docentes y deportivas, y\\nnÃºmero de puestos escolares, para impartir enseÃ±anzas con garantÃ\xada de calidad. Por su\\nparte, el artÃ\xadculo 23 de la misma ley condiciona la apertura y funcionamiento de los centros\\ndocentes privados al principio de auto

In [20]:
df['Documento'][1]

'BOE\\nBOLETÃ\x8dN OFICIAL DEL ESTADO\\nNÃºm. 62\\nViernes 12 de marzo de 2010\\nSec. I. PÃ¡g. 24831\\nI. DISPOSICIONES GENERALES\\nMINISTERIO DE EDUCACIÃ“N\\n4132\\nReal Decreto 132/2010, de 12 de febrero, por el que se establecen los requisitos\\nmÃ\xadnimos de los centros que impartan las enseÃ±anzas del segundo ciclo de la\\neducaciÃ³n infantil, la educaciÃ³n primaria y la educaciÃ³n secundaria.\\nLa Ley OrgÃ¡nica 8/1985, de 3 de julio, reguladora del Derecho a la EducaciÃ³n,\\nestablece, en su artÃ\xadculo 14, que todos los centros docentes, independientemente de su\\ntitularidad, deberÃ¡n reunir unos requisitos mÃ\xadnimos referidos a titulaciÃ³n acadÃ©mica del\\nprofesorado, relaciÃ³n numÃ©rica alumno-profesor, instalaciones docentes y deportivas, y\\nnÃºmero de puestos escolares, para impartir enseÃ±anzas con garantÃ\xada de calidad. Por su\\nparte, el artÃ\xadculo 23 de la misma ley condiciona la apertura y funcionamiento de los centros\\ndocentes privados al principio de auto

In [21]:
file_2

<_io.TextIOWrapper name='boe.txt' mode='r' encoding='utf8'>

In [22]:
with open('boe.txt', encoding='Latin1') as f:
    contents = f.read()
    print(contents)

BOE\nBOLETÃÂN OFICIAL DEL ESTADO\nNÃÂºm. 62\nViernes 12 de marzo de 2010\nSec. I. PÃÂ¡g. 24831\nI. DISPOSICIONES GENERALES\nMINISTERIO DE EDUCACIÃâN\n4132\nReal Decreto 132/2010, de 12 de febrero, por el que se establecen los requisitos\nmÃÂ­nimos de los centros que impartan las enseÃÂ±anzas del segundo ciclo de la\neducaciÃÂ³n infantil, la educaciÃÂ³n primaria y la educaciÃÂ³n secundaria.\nLa Ley OrgÃÂ¡nica 8/1985, de 3 de julio, reguladora del Derecho a la EducaciÃÂ³n,\nestablece, en su artÃÂ­culo 14, que todos los centros docentes, independientemente de su\ntitularidad, deberÃÂ¡n reunir unos requisitos mÃÂ­nimos referidos a titulaciÃÂ³n acadÃÂ©mica del\nprofesorado, relaciÃÂ³n numÃÂ©rica alumno-profesor, instalaciones docentes y deportivas, y\nnÃÂºmero de puestos escolares, para impartir enseÃÂ±anzas con garantÃÂ­a de calidad. Por su\nparte, el artÃÂ­culo 23 de la misma ley condiciona la apertura y funcionamiento de los centros\ndocentes privados al principi

##### https://www.simplifiedpython.net/pdf-to-text-python-extract-text-from-pdf-documents-using-pypdf2-module/



In [23]:
pdfFileObject = open(r"BOE-A-2010-4132.pdf", 'rb')

In [24]:
pdfReader = PyPDF2.PdfFileReader(pdfFileObject)

In [25]:
pdfReader

In [26]:
print(" No. Of Pages :", pdfReader.numPages)

 No. Of Pages : 10


In [27]:
pageObject = pdfReader.getPage(0)
 
print(pageObject.extractText())

BOLETÍN OFICIAL DEL ESTADO
Núm. 62 Viernes 12 de marzo de 2010
 Sec. I.   Pág. 24831
I. DISPOSICIONES GENERALESMINISTERIO DE EDUCACIÓN4132Real Decreto 132/2010, de 12 de febrero, por el que se establecen los requisitos 
mínimos de los centros que impartan las enseñanzas del segundo ciclo de la 

educación infantil, la educación primaria y la educación secundaria.La Ley Orgánica 8/1985, de 3 de julio, reguladora del Derecho a la Educación, 
establece, en su artículo 14, que todos los centros docentes, independientemente de su 

titularidad, deberán reunir unos requisitos mínimos referidos a titulación académica del 

profesorado, relación numérica alumno-profesor, instalaciones docentes y deportivas, y 

número de puestos escolares, para impartir enseñanzas con garantía de calidad. Por su 

parte, el artículo 23 de la misma ley condiciona la apertura y funcionamiento de los centros 

docentes privados al principio de autorización administrativa que se concederá siempre 

que aquellos re

In [29]:
# https://www.tutorialspoint.com/working-with-pdf-files-in-python

# with open('BOE-A-2010-4132.pdf', 'rb') as f:
#     pdf = PyPDF2.PdfFileReader(f)
#     info = pdf.getDocumentInfo()
#     number_of_pages = pdf.getNumPages()

# print("Author: \t", info.author)
# print()
# print("Creator: \t", info.creator)
# print()
# print("Producer: \t",info.producer)
# print()
# print("Subject: \t", info.subject)
# print()
# print("title: \t",info.title)
# print()
# print("Number of Pages in pdf: \t",number_of_pages)

# prueba = ''
# for i in range(pdf.getNumPages()):
#     prueba = prueba + pdf.getPage(i).extractText()
#     print(len(prueba))
    
# pdf.getPage(2)

In [30]:
# pdf.getPage(2).extractText()

In [31]:
# with open('prueba_es.pdf', 'rb') as f:
#     pdf = PyPDF2.PdfFileReader(f)
#     info = pdf.getDocumentInfo()
#     number_of_pages = pdf.getNumPages()
# print("Author: \t", info.author)
# print()
# print("Creator: \t", info.creator)
# print()
# print("Producer: \t",info.producer)
# print()
# print("Subject: \t", info.subject)
# print()
# print("title: \t",info.title)
# print()
# print("Number of Pages in pdf: \t",number_of_pages)

In [32]:
# pdfFileObject = open('BOE-A-2010-4132.pdf', 'rb')
# pdfReader = PyPDF2.PdfFileReader(pdfFileObject)
# prueba = ''
# for i in range(pdfReader.getNumPages()):
#     prueba = prueba + pdfReader.getPage(i).extractText()
#     print(len(prueba))
# prueba = prueba.replace('\n',' ')

In [33]:
# pdfReader.getPage(1).extractText()

In [35]:
# prueba

In [36]:
pdfReader.getDocumentInfo()

{'/Creator': 'eBOE',
 '/Producer': 'Adobe PDF Library 8.0',
 '/Title': 'Disposición 4132 del BOE núm. 62 de 2010',
 '/Keywords': 'DECRETO 132/2010 de 12/02/2010;MINISTERIO DE EDUCACIÓN;BOE-A-2010-4132;BOE 62 de 2010;4132;12/03/2010',
 '/ModDate': "D:20100311232857+01'00'",
 '/Trapped': '/False',
 '/Subject': 'BOE-A-2010-4132',
 '/CreationDate': "D:20100311194246+01'00'",
 '/Author': 'MINISTERIO DE EDUCACIÓN'}

In [37]:
documento = 'BOE-A-2010-4132.pdf'

with open(documento, 'rb') as f:
    pdf = PyPDF2.PdfFileReader(f)
    info = pdf.getDocumentInfo()
    number_of_pages = pdf.getNumPages()

# text = ''
# for i in range(pdf.getNumPages()):
#     text = text + pdf.getPage(i).extractText()
#     print(len(prueba))

In [38]:
# Incluir el documento en PDF
# --------------------------------------------------------------------

def resume_pdfs(documento, alpha):
    
    pdfFileObject = open(documento, 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObject)
    text = ''
    for i in range(pdfReader.getNumPages()):
        text = text + pdfReader.getPage(i).extractText()
    text = text.replace('\n',' ')
#     print(text)
#         print(len(prueba))
      
    
    sentences = sent_tokenize(text)
    total_documents = len(sentences)
    #print(sentences)

    # 2 Create the Frequency matrix of the words in each sentence.
    freq_matrix = _create_frequency_matrix(sentences)
    #print(freq_matrix)

    '''
    Term frequency (TF) is how often a word appears in a document, divided by how many words are there in a document.
    '''
    # 3 Calculate TermFrequency and generate a matrix
    tf_matrix = _create_tf_matrix(freq_matrix)
    #print(tf_matrix)

    # 4 creating table for documents per words
    count_doc_per_words = _create_documents_per_words(freq_matrix)
    #print(count_doc_per_words)

    '''
    Inverse document frequency (IDF) is how unique or rare a word is.
    '''
    # 5 Calculate IDF and generate a matrix
    idf_matrix = _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents)
    #print(idf_matrix)

    # 6 Calculate TF-IDF and generate a matrix
    tf_idf_matrix = _create_tf_idf_matrix(tf_matrix, idf_matrix)
    #print(tf_idf_matrix)

    # 7 Important Algorithm: score the sentences
    sentence_scores = _score_sentences(tf_idf_matrix)
    #print(sentence_scores)

    # 8 Find the threshold
    threshold = _find_average_score(sentence_scores)
    #print(threshold)

    # 9 Important Algorithm: Generate the summary (alpha between 0.5 - 1.5)
    summary = _generate_summary(sentences, sentence_scores, alpha * threshold)
    
    
    print("Título: \t",pdfReader.getDocumentInfo().title)
    print('--------------------------------------------------------------------')
    print("Número de páginas: {} \nLongitud del documento original: {}\nLongitud del resumen: {}".format(number_of_pages, len(text), len(summary)))
    print('--------------------------------------------------------------------')
    print(summary)

In [40]:
resume_pdfs('prueba_es.pdf', 0.8)

Título: 	 Proyecciones macroeconómicas de la economía española (2021-2023)
--------------------------------------------------------------------
Número de páginas: 10 
Longitud del documento original: 6618
Longitud del resumen: 2324
--------------------------------------------------------------------
                      1 / 2   23 . A su vez,  estas últimas   dependerán de  dos aspectos. En primer  lugar, de  las  secuelas   que  la pandemia   pueda dejar sobre el tejido productivo y laboral ,   en  términos de destrucción de empresas y desempleo de larga duración . El  escenario central   contempla la posibilidad de que en los próximos meses surjan nuevos  brotes de la enfermedad, cuya contención requeriría aplicar  medidas de contención  similares a las vigentes  recientemente . Se proyecta, además, que la progresiva  administración de las vacunas permita una retirada gradual de  estas  medidas, hasta su  desaparición   prácticamente   plena a finales de 2021. Asim ismo , la natural

In [41]:
import textract

ModuleNotFoundError: No module named 'textract'